# Relatório de Avaliação de Modelos para Armazenamento de Áudio em Bases de Dados Vetoriais (Milvus)

Este relatório apresenta uma avaliação comparativa (benchmark) de diferentes modelos utilizados processar e armazenar ficheiros de áudio em bases de dados vetoriais (Milvus). Ao longo deste notebook interativo, será possível analisar o desempenho de cada modelo/embedding, permitindo perceber qual deles se adapta melhor às nossas necessidades. No final, será possível eleger o modelo ou embedding mais adequado para o nosso caso específico.


## Introdução

## Falar o que fizemos para fazer o benchmark

## Fases do projeto

## Porque o milvus?

-TODO: Falar porque é que escolhemos o milvus

### Falar dos embeddings escolhidos

### Falar do script de benchmark...

## Análise dos resultados

Instalar dependências
```bash
pip install -r requirements.txt
```

```text
# requirements.txt

TODO ESCREVER O QUE ESTÁ NO REQUERIMENTS.TXT
```

PARA CORRER O NOTEBOOK É NECESSÁRIO TER CERTAS DEPÊNCIAS... ALSO O MILVUS NÃO VAI DAR... MAS É POSSIVEL FAZER A INSTALAÇÃO DO MILVUS FACILMENTE...
É SÓ CORRER O DOCKER COM O COMANDO ... 

TODO : VER FORMAS DE FAZER BUILD DO FICHEIRO DOCKER, PARA CONSEGUIR LEVANTAR A BASE DE DADOS... VER TAMBÉM SE OS FICHEIROS SÃO PERSISTENTES... EU ACHO QUE SIM


In [16]:
!pip install -r requirements_jupyter.txt

### 1º Imports

In [17]:
# 📦 Importações e leitura dos dados
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from sklearn.manifold import TSNE
import ast
from pymilvus import MilvusClient
import librosa
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import soundfile as sf
import io
from IPython.display import display
import ipywidgets as widgets




### 2º Carregar dados

DEPOIS COMO VAMOS TER ESCALIBIDADE VAMOS TER QUE ADICIONAR O RESTO DOS BENCHMARKS

In [18]:
# 📂 Leitura e limpeza dos dados
try:
    df = pd.read_csv('benchmarks/benchmark_250/resultados_benchmark.csv')
    df['embedding'] = df['embedding'].apply(ast.literal_eval)
    
    # Success message with dataset information
    print(f"✅ Dados carregados com sucesso!")
    print(f"   - {len(df)} registros de embeddings carregados")
    print(f"   - Modelos disponíveis: {', '.join(df['modelo'].unique())}")
    
except FileNotFoundError as e:
    print(f"❌ Erro ao carregar os arquivos de benchmark: {e}")
    print("   Verifique se os arquivos estão no diretório correto: benchmarks/benchmark_30/")
except Exception as e:
    print(f"❌ Erro ao processar os dados: {e}")
    print("   Verifique o formato dos arquivos CSV e tente novamente.")

✅ Dados carregados com sucesso!
   - 1500 registros de embeddings carregados
   - Modelos disponíveis: wav2vec2, vggish, openl3, yamnet, clap, ast


### 3ºCarregar dados

In [19]:
# 🔄 Conexão com Milvus e seleção da coleção
try:
    # Tentar conectar ao Milvus
    client = MilvusClient(uri='http://localhost:19530')
    
    # Obter coleções disponíveis
    colecoes_disponiveis = client.list_collections()
    
    if colecoes_disponiveis:
        print(f"✅ Conexão com Milvus estabelecida com sucesso!")
        print(f"   - {len(colecoes_disponiveis)} coleções disponíveis")
        print(f"   - Coleções: {', '.join(colecoes_disponiveis)}")
        
    else:
        print("⚠️ Conectado ao Milvus, mas nenhuma coleção encontrada.")
        print("   Verifique se os dados foram carregados corretamente.")
        
except ConnectionError as e:
    print(f"❌ Erro de conexão com o Milvus: {e}")
    print("   Verifique se o servidor Milvus está em execução em http://localhost:19530")
    print("   Para iniciar o Milvus localmente com Docker, execute:")
    print("   docker-compose up -d")
    client = None
    
except Exception as e:
    print(f"❌ Erro ao conectar ao Milvus: {e}")
    print("   Verifique sua configuração e tente novamente.")
    client = None

✅ Conexão com Milvus estabelecida com sucesso!
   - 33 coleções disponíveis
   - Coleções: audio_wav2vec2_25, audio_ast_25, audio_openl3_30, audio_yamnet_25, audio_clap_25, audio_wav2vec2_50, audio_ast_50, audio_vggish_25, audio_yamnet_100, audio_ast_100, audio_clap_250, audio_ast_250, audio_openl3_50, audio_vggish_5, audio_ast_5, audio_vggish_50, audio_clap_50, audio_wav2vec2_100, audio_vggish_250, audio_openl3_25, audio_wav2vec2_5, audio_clap_5, audio_wav2vec2_30, audio_yamnet_50, audio_openl3_100, audio_yamnet_5, audio_vggish_30, audio_wav2vec2_250, audio_yamnet_250, audio_openl3_5, audio_vggish_100, audio_clap_100, audio_openl3_250


### 4º Analise dos resultados

### Base de dados com 5 ficheiros de audio


Fazer para os outros

### ANALISAR E EXPLICAR O QUE É QUE ESTÁ A ACONTECER.. EXPLICAR CADA UMA DAS TABELAS 

In [20]:
# 📊 Gráficos interativos por métrica em abas
tab_contents = ['Extração', 'Inserção', 'Boxplot Extração', 'Boxplot Inserção','Tamanho vs Extração','Tamanho vs Inserção' ]
children = []

fig1 = px.bar(df, x='modelo', y='tempo_extracao', color='modelo', title='Tempo de Extração por Modelo')
fig2 = px.bar(df, x='modelo', y='tempo_insercao', color='modelo', title='Tempo de Inserção por Modelo')
fig3 = px.box(df, x='modelo', y='tempo_extracao', color='modelo', title='Boxplot Tempo de Extração')
fig4 = px.box(df, x='modelo', y='tempo_insercao', color='modelo', title='Boxplot Tempo de Inserção')
fig5 = px.scatter(df, x='tamanho_audio_mb', y='tempo_extracao', color='modelo', title='Tamanho vs Extração')
fig6 = px.scatter(df, x='tamanho_audio_mb', y='tempo_insercao', color='modelo', title='Tamanho vs Inserção')

#ADICIONAR MAIS ? 

for fig in [fig1, fig2, fig3, fig4,fig5, fig6]:
    # Create a new tab for each figure
    out = widgets.Output()
    with out:
        display(fig)
    children.append(out)

tab = widgets.Tab(children=children)
for i in range(len(tab_contents)):
    tab.set_title(i, tab_contents[i])
display(tab)

### 5º RANKINGS... TEMPO DE INSERÇÃO ETC... FAZER ISTO PARA CADA COLECTION... 5, 25, 50, 100, 200 FICHEIROS...

EXPLICAR COMO É QUE A ESCALIBIDADE INFLUENCIA... CHEGAR A UMA CONCLUSÃO FINAL

### 7º EXPLICAR EVOLUÇÃO DOS RECURSOS

In [21]:
# 📈 Evolução dos recursos (CPU, RAM)
recursos = ['cpu_percent_after', 'ram_percent_after']
for recurso in recursos:
    fig = go.Figure()
    for modelo in df['modelo'].unique():
        subset = df[df['modelo'] == modelo]
        fig.add_trace(go.Scatter(x=subset.index, y=subset[recurso], mode='lines+markers', name=modelo))
    fig.update_layout(title=f'Evolução de {recurso.upper()}', xaxis_title='Index', yaxis_title=recurso)
    fig.show()

# FAZER RANKING DOS RECURSOS... CHEGAR A UMA CONCLUSÃO FINAL

## Explicação do Sistema de Ranking dos Modelos de Embeddings
O sistema de ranking implementado no notebook oferece uma visão comparativa detalhada sobre o desempenho dos diferentes modelos de embeddings de áudio. Vamos explicar o significado dos valores apresentados e como interpretá-los:

Rankings Individuais por Métrica:
- Tempo de Extração (s)
Representa quanto tempo cada modelo leva para processar o áudio e gerar o embedding
Valores menores são melhores (indicam processamento mais rápido)
Um modelo com tempo de extração baixo é ideal para aplicações em tempo real ou processamento em lote de grandes coleções de áudio
- Tempo de Inserção (s)
Mede quanto tempo leva para armazenar o embedding na base de dados Milvus
Valores menores são melhores
Geralmente menos crítico que o tempo de extração, mas importante para fluxos de ingestão de dados em grande escala
- Pico de Memória (MB)
Indica a quantidade máxima de memória RAM utilizada durante o processamento
Valores menores são melhores (menos recursos consumidos)
Crítico para ambientes com recursos limitados ou aplicações que precisam processar muitos arquivos simultaneamente
- Uso de CPU (%)
Porcentagem de utilização da CPU durante o processamento
Valores menores são melhores
Modelos com menor uso de CPU deixam mais recursos disponíveis para outras tarefas
- Uso de RAM (%)
Percentual total de memória do sistema utilizada
Valores menores são melhores
Diferente do "Pico de Memória" que é específico do processo
- Dimensionalidade
Tamanho do vetor de embedding gerado
Não é classificado como "melhor" ou "pior" diretamente

Embeddings maiores (maior dimensionalidade) podem capturar mais informações, mas:
Consomem mais espaço de armazenamento
Podem ser mais lentos para pesquisa
Podem exigir mais recursos computacionais
Ranking Geral
Pontuação (0-100)
Representa uma avaliação holística do desempenho do modelo
Valores maiores são melhores (mais próximo de 100 = desempenho ideal)
Calculada usando uma média ponderada normalizada das métricas principais
Pesos das Métricas no Ranking Geral
O ranking geral usa a seguinte distribuição de pesos:

Tempo de extração: 35% (mais peso devido à sua importância crítica)
Pico de memória: 25% (segundo mais importante)
Uso de CPU: 20% (impacto moderado)
Tempo de inserção: 15% (menos crítico que a extração)
Uso de RAM total: 5% (menor peso pois depende de outros fatores do sistema)
Processo de Normalização
Para criar um ranking justo, o sistema:

Normaliza cada métrica para uma escala de 0-1 (onde 0 é o melhor valor)
Aplica os pesos definidos para cada métrica
Inverte a escala final para que valores maiores (próximos de 100) sejam melhores
Interpretação dos Resultados
Primeiro lugar (ouro): Representa o modelo com melhor equilíbrio entre velocidade, eficiência de recursos e desempenho
Segundo lugar (prata): Bom desempenho geral, possível alternativa ao primeiro colocado
Terceiro lugar (bronze): Desempenho aceitável, pode ser útil em cenários específicos
Demais posições: Podem ser adequadas para casos de uso específicos, dependendo da prioridade (velocidade vs. qualidade vs. recursos)
É importante notar que o ranking geral oferece uma visão equilibrada, mas a escolha do melhor modelo ainda depende das prioridades específicas do seu projeto. Por exemplo, se você tem restrições severas de memória, pode preferir um modelo que não está no topo do ranking geral, mas que tem excelente desempenho na métrica de consumo de memória.

In [22]:
# 🏆 Ranking dos modelos por diferentes métricas
def create_ranking_table(df, metric_column, metric_name, lower_is_better=True):
    """
    Cria uma tabela de classificação dos modelos baseada numa métrica específica.
    
    Args:
        df: DataFrame com os resultados do benchmark
        metric_column: Nome da coluna da métrica a ser analisada
        metric_name: Nome amigável da métrica para exibição
        lower_is_better: Se True, valores menores são melhores (ex: tempo, memória)
    
    Returns:
        DataFrame com o ranking dos modelos
    """
    # Calcular média e desvio padrão para cada modelo
    ranking = df.groupby('modelo')[metric_column].agg(['mean', 'std', 'min', 'max']).reset_index()
    
    # Renomear colunas
    ranking.columns = ['Modelo', f'Média {metric_name}', f'Desvio Padrão', f'Mínimo', f'Máximo']
    
    # Ordenar resultados (crescente se lower_is_better=True, decrescente caso contrário)
    ranking = ranking.sort_values(by=f'Média {metric_name}', ascending=lower_is_better)
    
    # Adicionar coluna de ranking
    ranking.insert(0, 'Posição', range(1, len(ranking) + 1))
    
    return ranking

# Criar rankings para diferentes métricas
try:
    print("🏆 Rankings dos Modelos de Embeddings de Áudio")
    
    # Definir as métricas e seus nomes amigáveis
    metrics = [
        ('tempo_extracao', 'Tempo de Extração (s)', True),
        ('tempo_insercao', 'Tempo de Inserção (s)', True),
        ('memoria_peak_mb', 'Pico de Memória (MB)', True),
        ('cpu_percent_after', 'Uso de CPU (%)', True),
        ('ram_percent_after', 'Uso de RAM (%)', True),
        ('gpu_usage', 'Uso de GPU (%)', True),
        ('dimensao', 'Dimensionalidade', None)  # None porque não é melhor nem pior
    ]
    
    # Criar abas para os diferentes rankings
    tab_contents = [m[1] for m in metrics]
    children = []
    
    for metric_col, metric_name, lower_is_better in metrics:
        if metric_col in df.columns:
            # Criar tabela de ranking
            if lower_is_better is None:  # Para dimensionalidade, só mostramos os valores sem julgar
                ranking = df.groupby('modelo')[metric_col].first().reset_index()
                ranking.columns = ['Modelo', metric_name]
                ranking = ranking.sort_values(by=metric_name)
                ranking.insert(0, 'Posição', range(1, len(ranking) + 1))
            else:
                ranking = create_ranking_table(df, metric_col, metric_name, lower_is_better)
            
            # Criar visualização de tabela colorida com Plotly
            fig = go.Figure(data=[go.Table(
                header=dict(
                    values=list(ranking.columns),
                    fill_color='royalblue',
                    font=dict(color='white', size=12),
                    align='left'
                ),
                cells=dict(
                    values=[ranking[col] for col in ranking.columns],
                    fill_color=[['lightcyan' if i % 2 == 0 else 'white' for i in range(len(ranking))]],
                    align='left',
                    font_size=11,
                    height=25
                )
            )])
            
            fig.update_layout(
                title=f"Ranking por {metric_name}",
                height=400,
                margin=dict(l=20, r=20, t=40, b=20)
            )
            
            # Adicionar à aba
            out = widgets.Output()
            with out:
                display(fig)
            children.append(out)
        else:
            # Se a métrica não existe, mostrar mensagem
            out = widgets.Output()
            with out:
                print(f"⚠️ Métrica '{metric_name}' não disponível nos dados do benchmark.")
            children.append(out)
    
    # Criar widget de abas para os rankings
    ranking_tab = widgets.Tab(children=children)
    for i in range(len(tab_contents)):
        ranking_tab.set_title(i, tab_contents[i])
    
    # Mostrar widget
    display(ranking_tab)
    
    # Criar um ranking geral (média ponderada das principais métricas)
    print("\n🏆 Ranking Geral dos Modelos")
    
    # Definir pesos para as métricas principais (ajuste conforme necessidade do projeto)
    weights = {
        'tempo_extracao': 0.35,  # Tempo de extração é muito importante
        'memoria_peak_mb': 0.25,  # Uso de memória é importante
        'cpu_percent_after': 0.20,  # Uso de CPU tem peso médio
        'tempo_insercao': 0.15,  # Tempo de inserção é menos importante
        'ram_percent_after': 0.05  # RAM total tem menos peso
    }
    
    # Verificar quais métricas estão disponíveis
    available_metrics = [m for m in weights.keys() if m in df.columns]
    
    if len(available_metrics) >= 3:  # Exigir pelo menos 3 métricas para um ranking geral adequado
        # Normalizar os pesos para as métricas disponíveis
        total_weight = sum(weights[m] for m in available_metrics)
        normalized_weights = {m: weights[m]/total_weight for m in available_metrics}
        
        # Calcular médias para cada modelo e métrica
        model_metrics = {}
        for model in df['modelo'].unique():
            model_data = df[df['modelo'] == model]
            model_metrics[model] = {}
            
            for metric in available_metrics:
                model_metrics[model][metric] = model_data[metric].mean()
        
        # Normalizar valores entre 0-1 (onde 0 é melhor, 1 é pior)
        normalized_metrics = {}
        for metric in available_metrics:
            min_val = min(model_metrics[m][metric] for m in model_metrics)
            max_val = max(model_metrics[m][metric] for m in model_metrics)
            range_val = max_val - min_val
            
            if range_val > 0:  # Evitar divisão por zero
                for model in model_metrics:
                    if metric not in normalized_metrics:
                        normalized_metrics[metric] = {}
                    normalized_metrics[metric][model] = (model_metrics[model][metric] - min_val) / range_val
            else:
                # Se todos os valores forem iguais, atribuir 0.5 (neutro)
                for model in model_metrics:
                    if metric not in normalized_metrics:
                        normalized_metrics[metric] = {}
                    normalized_metrics[metric][model] = 0.5
        
        # Calcular pontuação final (ponderada)
        final_scores = {}
        for model in model_metrics:
            score = 0
            for metric in available_metrics:
                score += normalized_metrics[metric][model] * normalized_weights[metric]
            final_scores[model] = score
        
        # Criar DataFrame com ranking final
        final_ranking = pd.DataFrame({
            'Modelo': list(final_scores.keys()),
            'Pontuação Final': list(final_scores.values())
        })
        
        # Calcular pontuação de 0-100 (inverter para que maior seja melhor)
        final_ranking['Pontuação (0-100)'] = 100 * (1 - final_ranking['Pontuação Final'])
        
        # Ordenar por pontuação (maior é melhor)
        final_ranking = final_ranking.sort_values('Pontuação (0-100)', ascending=False)
        final_ranking.insert(0, 'Posição', range(1, len(final_ranking) + 1))
        
        # Criar visualização de tabela colorida para o ranking final
        fig = go.Figure(data=[go.Table(
            header=dict(
                values=['Posição', 'Modelo', 'Pontuação (0-100)'],
                fill_color='darkblue',
                font=dict(color='white', size=14),
                align='center'
            ),
            cells=dict(
                values=[
                    final_ranking['Posição'],
                    final_ranking['Modelo'],
                    final_ranking['Pontuação (0-100)'].round(2)
                ],
                fill_color=[
                    ['gold' if pos == 1 else 'silver' if pos == 2 else 'lightcoral' if pos == 3 
                     else 'white' for pos in final_ranking['Posição']]
                ],
                align='center',
                font_size=13,
                height=30
            )
        )])
        
        fig.update_layout(
            title="🏆 Ranking Geral dos Modelos de Embeddings de Áudio",
            height=400,
            margin=dict(l=20, r=20, t=50, b=20)
        )
        
        display(fig)
        
        # Criar gráfico de barras para o ranking final
        bar_fig = px.bar(
            final_ranking, 
            x='Modelo', 
            y='Pontuação (0-100)', 
            title='Comparação Geral dos Modelos', 
            color='Pontuação (0-100)',
            color_continuous_scale=px.colors.sequential.Viridis,
            text='Pontuação (0-100)'
        )
        
        bar_fig.update_traces(texttemplate='%{text:.1f}', textposition='outside')
        bar_fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
        
        display(bar_fig)
        
    else:
        print("⚠️ Dados insuficientes para gerar um ranking geral significativo.")

except Exception as e:
    import traceback
    print(f"❌ Erro ao gerar ranking dos modelos: {e}")
    traceback.print_exc()

🏆 Rankings dos Modelos de Embeddings de Áudio



🏆 Ranking Geral dos Modelos


EXPLICAR PARA A ESCALIBIDADE

In [23]:
# 📊 Comparação de benchmarks com diferentes quantidades de ficheiros
from IPython.display import display, clear_output

def carregar_multiplos_benchmarks():
    """Carrega todos os benchmarks disponíveis em uma estrutura unificada"""
    tamanhos = [5, 25, 50, 100, 250]
    dataframes = {}
    
    for tamanho in tamanhos:
        try:
            caminho = f'benchmarks/benchmark_{tamanho}/resultados_benchmark.csv'
            df = pd.read_csv(caminho)
            df['embedding'] = df['embedding'].apply(ast.literal_eval)
            df['tamanho_benchmark'] = tamanho  # Adiciona coluna identificando o tamanho
            dataframes[tamanho] = df
            print(f"✅ Benchmark com {tamanho} ficheiros carregado: {len(df)} registros")
        except FileNotFoundError:
            print(f"⚠️ Benchmark com {tamanho} ficheiros não encontrado")
        except Exception as e:
            print(f"❌ Erro ao carregar benchmark com {tamanho} ficheiros: {e}")
    
    return dataframes

# Função para exibir visualizações de um benchmark específico
def display_benchmark(df):
    """Exibe visualizações para um benchmark específico"""
    if df is None or df.empty:
        print("Sem dados para exibir")
        return
    
    # Limpar saída anterior
    clear_output(wait=True)
    
    # Mostrar informações básicas
    print(f"## Benchmark com {df['tamanho_benchmark'].iloc[0]} ficheiros")
    print(f"- Total de registros: {len(df)}")
    print(f"- Modelos disponíveis: {', '.join(df['modelo'].unique())}")
    
    # Visualizações principais
    fig1 = px.bar(df, x='modelo', y='tempo_extracao', color='modelo', 
                 title=f'Tempo de Extração por Modelo (Base: {df["tamanho_benchmark"].iloc[0]} ficheiros)')
    fig2 = px.bar(df, x='modelo', y='tempo_insercao', color='modelo', 
                 title=f'Tempo de Inserção por Modelo (Base: {df["tamanho_benchmark"].iloc[0]} ficheiros)')
    
    display(fig1)
    display(fig2)
    
    # Mostrar estatísticas
    stats = df.groupby('modelo')[['tempo_extracao', 'tempo_insercao']].agg(['mean', 'std']).reset_index()
    display(stats)

# Carregar todos os benchmarks
benchmarks = carregar_multiplos_benchmarks()

# Interface para seleção do benchmark a visualizar
tamanhos_disponiveis = list(benchmarks.keys())
if tamanhos_disponiveis:
    # Definir a função de atualização de visualização
    def atualizar_visualizacao(change):
        tamanho_selecionado = change['new']
        if tamanho_selecionado in benchmarks:
            display_benchmark(benchmarks[tamanho_selecionado])
    
    # Criar o seletor
    selector = widgets.Dropdown(
        options=tamanhos_disponiveis,
        description='Benchmark:',
        value=tamanhos_disponiveis[0]
    )
    
    selector.observe(atualizar_visualizacao, names='value')
    display(selector)
    
    # Mostrar o benchmark inicial
    display_benchmark(benchmarks[tamanhos_disponiveis[0]])
else:
    print("Nenhum benchmark encontrado.")

# Análise de escalabilidade
if len(benchmarks) >= 2:
    print("\n# Análise de Escalabilidade")
    print("Observe como cada modelo se comporta com o aumento do número de ficheiros:")
    
    # 1. Gráfico de evolução de tempo de extração com o aumento do número de arquivos
    metricas_escalabilidade = [
        ('tempo_extracao', 'Tempo Médio de Extração (s)'),
        ('tempo_insercao', 'Tempo Médio de Inserção (s)'),
        ('memoria_peak_mb', 'Pico Médio de Memória (MB)'),
        ('cpu_percent_after', 'Uso Médio de CPU (%)')
    ]
    
    tab_contents_escala = [m[1] for m in metricas_escalabilidade]
    children_escala = []
    
    for metrica, titulo in metricas_escalabilidade:
        # Criar figura
        fig_escala = go.Figure()
        
        # Verificar se a métrica existe em pelo menos um dataframe
        if not any(metrica in df.columns for df in benchmarks.values()):
            out = widgets.Output()
            with out:
                print(f"Métrica '{titulo}' não disponível nos dados")
            children_escala.append(out)
            continue
        
        # Para cada modelo, coletar dados de todos os benchmarks
        all_models = set()
        for df in benchmarks.values():
            if not df.empty:
                all_models.update(df['modelo'].unique())
        
        for modelo in all_models:
            x_vals = []  # Tamanhos dos benchmarks
            y_vals = []  # Valores médios da métrica
            
            for tamanho, df in benchmarks.items():
                if not df.empty and modelo in df['modelo'].unique() and metrica in df.columns:
                    x_vals.append(tamanho)
                    y_vals.append(df[df['modelo'] == modelo][metrica].mean())
            
            # Ordenar por tamanho (para garantir que a linha seja traçada corretamente)
            pontos = sorted(zip(x_vals, y_vals))
            if pontos:
                x_vals, y_vals = zip(*pontos)
                fig_escala.add_trace(go.Scatter(
                    x=x_vals, 
                    y=y_vals, 
                    mode='lines+markers', 
                    name=modelo
                ))
        
        fig_escala.update_layout(
            title=f'Escalabilidade: {titulo} por Tamanho da Base',
            xaxis_title='Número de Ficheiros',
            yaxis_title=titulo,
            hovermode='closest'
        )
        
        # Adicionar à aba
        out = widgets.Output()
        with out:
            display(fig_escala)
        children_escala.append(out)
    
    # Criar widget de abas para os gráficos de escalabilidade
    escalabilidade_tab = widgets.Tab(children=children_escala)
    for i in range(len(tab_contents_escala)):
        escalabilidade_tab.set_title(i, tab_contents_escala[i])
    
    display(escalabilidade_tab)
    
    # 2. Comparação de rankings entre benchmarks
    print("\n## Evolução dos Rankings com a Escala")
    
    # Extrair rankings para cada tamanho
    rankings_por_tamanho = {}
    for tamanho, df in benchmarks.items():
        if not df.empty:
            # Usar algoritmo de ranking simplificado
            rankings = {}
            for modelo in df['modelo'].unique():
                # Cálculo do ranking (simplificado)
                metricas = {
                    'tempo_extracao': df[df['modelo'] == modelo]['tempo_extracao'].mean(),
                    'memoria_peak_mb': df[df['modelo'] == modelo]['memoria_peak_mb'].mean() if 'memoria_peak_mb' in df.columns else None,
                    'cpu_percent_after': df[df['modelo'] == modelo]['cpu_percent_after'].mean() if 'cpu_percent_after' in df.columns else None,
                    'tempo_insercao': df[df['modelo'] == modelo]['tempo_insercao'].mean() if 'tempo_insercao' in df.columns else None
                }
                # Pontuação simples (quanto menor, melhor)
                pontuacao = sum(v for v in metricas.values() if v is not None) 
                rankings[modelo] = pontuacao
            
            # Ordenar do melhor para o pior
            rankings_ordenados = {k: v for k, v in sorted(rankings.items(), key=lambda item: item[1])}
            rankings_por_tamanho[tamanho] = rankings_ordenados
    
    # Visualizar como os rankings mudam
    # Visualizar como os rankings mudam
if rankings_por_tamanho:
    # Criar tabela de evolução de posições
    modelos = set()
    for r in rankings_por_tamanho.values():
        modelos.update(r.keys())
    
    # Convert set to list for pandas DataFrame index
    modelos = sorted(list(modelos))
    
    tabela_posicoes = pd.DataFrame(index=modelos)
    
    for tamanho, ranking in sorted(rankings_por_tamanho.items()):
        # Converte o ranking em posições (1º, 2º, 3º...)
        posicoes = {modelo: pos+1 for pos, modelo in enumerate(ranking.keys())}
        tabela_posicoes[f'{tamanho} ficheiros'] = pd.Series(posicoes)
        
        # Mostrar tabela colorida
        fig_ranking = go.Figure(data=[go.Table(
            header=dict(
                values=['Modelo'] + [f'{t} ficheiros' for t in sorted(rankings_por_tamanho.keys())],
                fill_color='royalblue',
                font=dict(color='white', size=12),
                align='center'
            ),
            cells=dict(
                values=[tabela_posicoes.index] + [tabela_posicoes[col] for col in tabela_posicoes.columns],
                fill_color='lightcyan',
                align='center'
            )
        )])
        
        fig_ranking.update_layout(
            title="Evolução das Posições no Ranking por Escala",
            height=400,
            margin=dict(l=20, r=20, t=50, b=20)
        )
        
        display(fig_ranking)
        
        # Heatmap de estabilidade do ranking
        heatmap_data = tabela_posicoes.copy()
        
        # Normalizar valores para visualização (posição 1 = melhor = 1.0, posições piores = valores menores)
        for col in heatmap_data.columns:
            max_pos = heatmap_data[col].max()
            if pd.notna(max_pos) and max_pos > 1:
                heatmap_data[col] = 1 - (heatmap_data[col] - 1) / max_pos
        
        fig_heatmap = px.imshow(
            heatmap_data,
            labels=dict(x="Tamanho do Benchmark", y="Modelo", color="Posição Relativa"),
            x=heatmap_data.columns,
            y=heatmap_data.index,
            color_continuous_scale='viridis',
            title="Estabilidade dos Rankings com Escala"
        )
        
        display(fig_heatmap)
        
        # Análise de eficiência com escala
        print("\n## Eficiência com Escala")
        print("Esta análise mostra quais modelos mantêm bom desempenho mesmo com grandes volumes de dados:")
        
        # Calcular a tendência de cada modelo em tempo de extração com aumento da escala
        slopes = {}
        for modelo in modelos:
            x_vals = []
            y_vals = []
            for tamanho, df in benchmarks.items():
                if not df.empty and modelo in df['modelo'].unique() and 'tempo_extracao' in df.columns:
                    x_vals.append(tamanho)
                    y_vals.append(df[df['modelo'] == modelo]['tempo_extracao'].mean())
            
            if len(x_vals) >= 2:
                # Calcular tendência (aproximação simples)
                if max(x_vals) > min(x_vals):
                    slope = (max(y_vals) - min(y_vals)) / (max(x_vals) - min(x_vals))
                    slopes[modelo] = slope
        
        if slopes:
            slopes_df = pd.DataFrame({
                'Modelo': list(slopes.keys()),
                'Taxa de Crescimento': list(slopes.values())
            })
            
            slopes_df = slopes_df.sort_values('Taxa de Crescimento')
            
            fig_slopes = px.bar(
                slopes_df, 
                x='Modelo', 
                y='Taxa de Crescimento',
                title='Taxa de Crescimento do Tempo com Aumento da Base',
                color='Taxa de Crescimento',
                color_continuous_scale='RdYlGn_r'  # Vermelho = pior, Verde = melhor
            )
            
            display(fig_slopes)
            
            print("\nModelos com menor taxa de crescimento são mais escaláveis, pois seu tempo de processamento")
            print("aumenta mais lentamente com o crescimento do volume de dados.")
            
            # Adicionar interpretação dos resultados
            melhor_modelo = slopes_df.iloc[0]['Modelo']
            pior_modelo = slopes_df.iloc[-1]['Modelo']
            
            print(f"\n### Interpretação dos Resultados de Escalabilidade")
            print(f"- **{melhor_modelo}** demonstra a melhor escalabilidade, com menor taxa de crescimento de tempo.")
            print(f"- **{pior_modelo}** apresenta a escalabilidade mais desafiadora, com maior aumento de tempo conforme o volume de dados cresce.")
            print("\nEscolha o modelo considerando não apenas o desempenho atual, mas também a projeção futura do volume de dados.")

## Benchmark com 5 ficheiros
- Total de registros: 30
- Modelos disponíveis: wav2vec2, vggish, openl3, yamnet, clap, ast


modelo tempo_extracao           tempo_insercao          
                      mean       std           mean       std
0       ast       0.838650  0.332128       0.158832  0.129398
1      clap       0.229738  0.202934       0.053357  0.012093
2    openl3      27.741236  2.404783       0.082798  0.043130
3    vggish       0.285770  0.154726       0.048521  0.019274
4  wav2vec2       1.871834  0.884459       0.184742  0.243238
5    yamnet       0.150472  0.124131       0.044936  0.005034


# Análise de Escalabilidade
Observe como cada modelo se comporta com o aumento do número de ficheiros:



## Evolução dos Rankings com a Escala



## Eficiência com Escala
Esta análise mostra quais modelos mantêm bom desempenho mesmo com grandes volumes de dados:


...


Modelos com menor taxa de crescimento são mais escaláveis, pois seu tempo de processamento
aumenta mais lentamente com o crescimento do volume de dados.

### Interpretação dos Resultados de Escalabilidade
- **vggish** demonstra a melhor escalabilidade, com menor taxa de crescimento de tempo.
- **openl3** apresenta a escalabilidade mais desafiadora, com maior aumento de tempo conforme o volume de dados cresce.

Escolha o modelo considerando não apenas o desempenho atual, mas também a projeção futura do volume de dados.



## Eficiência com Escala
Esta análise mostra quais modelos mantêm bom desempenho mesmo com grandes volumes de dados:



Modelos com menor taxa de crescimento são mais escaláveis, pois seu tempo de processamento
aumenta mais lentamente com o crescimento do volume de dados.

### Interpretação dos Resultados de Escalabilidade
- **vggish** demonstra a melhor escalabilidade, com menor taxa de crescimento de tempo.
- **openl3** apresenta a escalabilidade mais desafiadora, com maior aumento de tempo conforme o volume de dados cresce.

Escolha o modelo considerando não apenas o desempenho atual, mas também a projeção futura do volume de dados.



## Eficiência com Escala
Esta análise mostra quais modelos mantêm bom desempenho mesmo com grandes volumes de dados:



Modelos com menor taxa de crescimento são mais escaláveis, pois seu tempo de processamento
aumenta mais lentamente com o crescimento do volume de dados.

### Interpretação dos Resultados de Escalabilidade
- **vggish** demonstra a melhor escalabilidade, com menor taxa de crescimento de tempo.
- **openl3** apresenta a escalabilidade mais desafiadora, com maior aumento de tempo conforme o volume de dados cresce.

Escolha o modelo considerando não apenas o desempenho atual, mas também a projeção futura do volume de dados.


Benefícios desta Abordagem
Análise de Escalabilidade: Mostra como cada modelo se comporta à medida que o número de arquivos aumenta

Evolução dos Rankings: Permite ver se os modelos que são melhores em conjuntos pequenos mantêm sua posição com conjuntos maiores

Estabilidade de Desempenho: O heatmap visualiza quais modelos são mais consistentes em diferentes escalas

Análise de Eficiência: Identifica quais modelos têm melhor "crescimento" de tempo/recursos, indicando escalabilidade

Interface Interativa: Permite alternar facilmente entre diferentes tamanhos de benchmark

Interpretação dos Resultados
Taxa de Crescimento: Modelos com menor taxa indicam melhor escalabilidade
Estabilidade do Ranking: Modelos que mantêm cores consistentes no heatmap são mais previsíveis em diferentes escalas
Evolução das Métricas: Os gráficos de linha mostram se o crescimento é linear, exponencial ou irregular
Esta análise permitirá determinar qual modelo é mais adequado não apenas para o tamanho atual da sua base, mas também para o crescimento futuro, facilitando decisões estratégicas sobre qual embedding adotar em produção.

### 8 VISUALIZAÇÕES DOS EMBEDDINGS NO ESPAÇO ... EXPLICAR O QUE SIGNIFICA SE ESTA MAIS PROXIMOS OU AFASTADOS UNS DOS OUTROS

In [24]:
# 🧠 Visualização t-SNE com validação
modelo_tsne = 'wav2vec2'
subset = df[df['modelo'] == modelo_tsne]
if len(subset) >= 2:
    try:
        emb = np.stack(subset['embedding'].values)
        tsne = TSNE(n_components=2, random_state=42, perplexity=min(30, len(emb)-1))
        emb_2d = tsne.fit_transform(emb)
        fig = px.scatter(x=emb_2d[:,0], y=emb_2d[:,1], text=subset['ficheiro'], title=f't-SNE ({modelo_tsne})')
        fig.show()
    except Exception as e:
        print(f'Erro no t-SNE: {e}')
else:
    print('Dados insuficientes para gerar t-SNE.')

### PROCURA REAL OU ADHOC NA BASE DE DADOS MILVUS... VAMOS USAR SEMPRE A BASE DE DADOS COM MAIS DADOS INSERIDOS... 

AD HOC SEARCH


-TODO FIX ADHOC SEARCH WITH YAMMET AND VGGISH

In [ ]:
# 📤 Pesquisa Ad-hoc (Upload ou Ficheiro da Base de Dados) e funções auxiliares
import tempfile
import librosa
import numpy as np
import torch
import pandas as pd
import plotly.express as px
import ast
import openl3
import io
import os

# Import transformers models
from transformers import (
    Wav2Vec2Processor, Wav2Vec2Model,
    ClapProcessor, ClapModel,
    ASTFeatureExtractor, ASTModel
)

import tensorflow as tf
import tensorflow_hub as hub

# Função para extrair embedding de um áudio
def extract_embedding(model_name, temp_path):
    audio, sr = librosa.load(temp_path, sr=None)
    
    if model_name == "wav2vec2":
        processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
        model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")
        # Resample if needed
        if sr != 16000:
            audio = librosa.resample(audio, orig_sr=sr, target_sr=16000)
        inputs = processor(audio, sampling_rate=16000, return_tensors="pt")
        with torch.no_grad():
            outputs = model(**inputs)
        return outputs.last_hidden_state.mean(dim=1).numpy()[0]
    
    elif model_name == "clap":
        processor = ClapProcessor.from_pretrained("laion/clap-htsat-unfused")
        model = ClapModel.from_pretrained("laion/clap-htsat-unfused")
        # Resample if needed
        if sr != 48000:
            audio = librosa.resample(audio, orig_sr=sr, target_sr=48000)
        inputs = processor(audios=audio, sampling_rate=48000, return_tensors="pt")
        with torch.no_grad():
            embeddings = model.get_audio_features(**inputs)
        return embeddings[0].numpy()
    
    elif model_name == "ast":
        fe = ASTFeatureExtractor.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")
        model = ASTModel.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")
        # Resample if needed
        if sr != 16000:
            audio = librosa.resample(audio, orig_sr=sr, target_sr=16000)
        inputs = fe(audio, sampling_rate=16000, return_tensors="pt")
        with torch.no_grad():
            outputs = model(**inputs)
        return outputs.last_hidden_state.mean(dim=1).numpy()[0]
    
    elif model_name == "vggish":
        # Mesmo código usado no Streamlit que funciona
        import tensorflow_hub as hub
        # Resample para 16kHz conforme exigido pelo VGGish
        if sr != 16000:
            audio = librosa.resample(audio, orig_sr=sr, target_sr=16000)
        # Converter para mono se for estéreo
        if len(audio.shape) > 1:
            audio = np.mean(audio, axis=1)
        # Carregar modelo e obter embedding
        vggish_model = hub.load('https://tfhub.dev/google/vggish/1')
        embedding = vggish_model(audio).numpy()
        # Fazer média se tivermos múltiplos embeddings
        if embedding.ndim > 1:
            embedding = np.mean(embedding, axis=0)
        return embedding
    
    elif model_name == "yamnet":
        # Mesmo código usado no Streamlit que funciona
        import tensorflow_hub as hub
        import tensorflow as tf
        # Resample para 16kHz conforme exigido pelo YAMNet
        if sr != 16000:
            audio = librosa.resample(audio, orig_sr=sr, target_sr=16000)
        # Converter para tensor do TensorFlow
        waveform = tf.convert_to_tensor(audio, dtype=tf.float32)
        # Carregar modelo e obter embedding
        yamnet_model = hub.load('https://tfhub.dev/google/yamnet/1')
        scores, embeddings, spectrogram = yamnet_model(waveform)
        # Fazer média dos embeddings se tivermos múltiplos
        embedding = np.mean(embeddings.numpy(), axis=0)
        return embedding
    
    elif model_name == "openl3":
        audio = np.mean(audio, axis=1) if audio.ndim > 1 else audio
        emb, _ = openl3.get_audio_embedding(audio, sr, content_type="music", embedding_size=512)
        return emb.mean(axis=0) if emb.ndim > 1 else emb.squeeze()
    
    else:
        raise ValueError("Modelo não suportado")

# Função para pesquisa real a partir do banco de dados
def real_search_from_db(df, client, modelo_milvus, collection_name, k):
    dimensoes_modelos = {
        "ast": 768, "clap": 512, "yamnet": 1024,
        "openl3": 512, "vggish": 128, "wav2vec": 768, "wav2vec2": 768,
    }
    dim_esperada = dimensoes_modelos.get(modelo_milvus)
    if not dim_esperada:
        raise ValueError(f"Dimensão desconhecida para modelo '{modelo_milvus}'")

    ficheiros = df[(df['modelo'] == modelo_milvus) & (df['embedding'].apply(lambda x: len(x) == dim_esperada))]['ficheiro'].tolist()
    if not ficheiros:
        raise ValueError(f"Sem ficheiros com dimensão {dim_esperada} para modelo '{modelo_milvus}'")

    row_query = df[(df['modelo'] == modelo_milvus) & (df['ficheiro'] == ficheiros[0])].iloc[0]
    embedding_query = row_query['embedding']

    result = client.search(
        collection_name=collection_name,
        data=[embedding_query],
        limit=k,
        output_fields=["id", "vector"]
    )
    hits = result[0] if result else []
    res_df = pd.DataFrame([{"id": h.get("id"), "distance": h.get("distance")} for h in hits])
    return res_df

# Interface de usuário
upload_widget = widgets.FileUpload(accept='.wav,.mp3,.flac,.ogg', multiple=False)
modelo_options = ["wav2vec2", "clap", "ast", "vggish", "yamnet", "openl3"]
modelo_widget = widgets.Dropdown(options=modelo_options, description="Modelo")
usar_ficheiro_existente = widgets.Checkbox(value=False, description="Usar ficheiro do dataset")
ficheiro_select = widgets.Dropdown(description="Ficheiro", options=[])

# Widget para seleção de coleção
collection_widget = widgets.Dropdown(description="Coleção", options=[])
usar_colecao_padrao = widgets.Checkbox(value=True, description="Usar coleção padrão")

output = widgets.Output()

# Função para atualizar opções de coleção
def update_collection_options(*args):
    try:
        collections = client.list_collections() if client else []
        if collections:
            collection_widget.options = collections
    except Exception as e:
        with output:
            print(f"Erro ao atualizar coleções: {e}")

# Função para atualizar opções de arquivo
def update_file_options(*args):
    if usar_ficheiro_existente.value:
        ficheiro_select.options = df[df['modelo'] == modelo_widget.value]['ficheiro'].tolist()
        ficheiro_select.layout.display = 'block'
    else:
        ficheiro_select.layout.display = 'none'
        # Exibe aviso para VGGish e YAMNet quando esses modelos são selecionados
        modelo_atual = modelo_widget.value

# Função para atualizar visibilidade do seletor de coleção
def update_collection_visibility(*args):
    if usar_colecao_padrao.value:
        collection_widget.layout.display = 'none'
    else:
        collection_widget.layout.display = 'block'
        # Atualize as opções quando o widget ficar visível
        update_collection_options()

def perform_search(b):
    with output:
        output.clear_output()
        try:
            query_id = None  # Initialize a variable to track query ID
            
            if usar_ficheiro_existente.value:
                if not ficheiro_select.value:
                    print("Por favor, selecione um ficheiro do dataset.")
                    return
                    
                # Usar ficheiro do dataset
                row = df[(df['modelo'] == modelo_widget.value) & (df['ficheiro'] == ficheiro_select.value)].iloc[0]
                embedding = np.array(row['embedding'])
                query_id = row.get('id')  # Get ID if available
                print(f"Usando embedding do ficheiro: {ficheiro_select.value}")
            
            else:
                # Este bloco estava dentro do if anterior, corrigindo a indentação
                if not upload_widget.value:
                    print("Por favor, carregue um ficheiro de áudio.")
                    return
                
                # Usar ficheiro carregado
                file_content = upload_widget.value[0]['content']
                file_bytes = io.BytesIO(file_content)
                file_name = upload_widget.value[0]['name']
                
                with tempfile.NamedTemporaryFile(delete=False, suffix=f"_{file_name}") as f:
                    f.write(file_bytes.read())
                    temp_path = f.name
                
                print(f"Extraindo embedding com modelo {modelo_widget.value}...")
                embedding = extract_embedding(modelo_widget.value, temp_path)
                os.unlink(temp_path)  # Remove o arquivo temporário
            
            # Determinar qual coleção usar
            if usar_colecao_padrao.value:
                collection_name = f"audio_{modelo_widget.value}"
            else:
                collection_name = collection_widget.value
                if not collection_name:
                    print("Por favor, selecione uma coleção.")
                    return
            
            # Verificar se a coleção existe
            if collection_name not in client.list_collections():
                print(f"Coleção {collection_name} não existe no Milvus!")
                return
                
            print(f"Pesquisando na coleção {collection_name}...")
            if query_id is not None:
                expr = f"id != {query_id}"
                result = client.search(
                    collection_name=collection_name,
                    data=[embedding.tolist()],
                    limit=6,  # Request one more result than needed
                    filter=expr,
                    output_fields=['id', 'vector']
                )
            else:
                result = client.search(
                    collection_name=collection_name,
                    data=[embedding.tolist()],
                    limit=6,  # Request one more to account for possible self-match
                    output_fields=['id', 'vector']
                )
            
            hits = result[0] if result else []
            if not hits:
                print("Nenhum resultado encontrado.")
                return
                
            df_res = pd.DataFrame([{'id': h.get('id'), 'distance': h.get('distance')} for h in hits])
            display(df_res)
            
            # Visualização dos resultados
            fig = px.bar(df_res, x='id', y='distance', title=f'Resultados da Pesquisa - Coleção: {collection_name}')
            fig.show()
            
        except NotImplementedError as e:
            print(f"⚠️ {str(e)}")
        except Exception as e:
            import traceback
            print(f"Erro ao processar ou pesquisar: {str(e)}")
            traceback.print_exc()

# Configuração da interface
search_button = widgets.Button(description="Pesquisar")
search_button.on_click(perform_search)
usar_ficheiro_existente.observe(update_file_options, names='value')
modelo_widget.observe(update_file_options, names='value')
usar_colecao_padrao.observe(update_collection_visibility, names='value')

# Inicializar visibilidade
ficheiro_select.layout.display = 'none'
collection_widget.layout.display = 'none'  # Esconde o seletor de coleção inicialmente

# Atualizar coleções disponíveis no início
update_collection_options()

# Exibir widgets
display(widgets.VBox([
    widgets.HBox([modelo_widget, usar_ficheiro_existente]),
    widgets.HBox([usar_colecao_padrao, collection_widget]),
    upload_widget,
    ficheiro_select,
    search_button,
    output
]))

# Inicializar opções de ficheiros
update_file_options()

BUG EXCLUIR PROPRIO ID .. ele esta a ir buscar o proprio

BUG ESCALABILIDADE